# Tutorial: Embedding Metadata for Improved Retrieval

## Setup

In [2]:
# %%bash
# pip install haystack-ai wikipedia sentence-transformers

## Indexing Documents with Metadata

Create a pipeline to store the small example dataset in the `InMemoryDocumentStore` with their embeddings. We will use `SentenceTransformersDocumentEmbedder` to generate embeddings for your `Documents` and write them to the document store with the `DocumentWriter`.

In [4]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice


def create_indexing_pipeline(document_store, metadata_fields_to_embed=None):
    document_cleaner = DocumentCleaner()
    document_splitter = DocumentSplitter(split_by="sentence", split_length=2)
    document_embedder = SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large", meta_fields_to_embed=metadata_fields_to_embed
    )
    document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("cleaner", document_cleaner)
    indexing_pipeline.add_component("splitter", document_splitter)
    indexing_pipeline.add_component("embedder", document_embedder)
    indexing_pipeline.add_component("writer", document_writer)

    indexing_pipeline.connect("cleaner", "splitter")
    indexing_pipeline.connect("splitter", "embedder")
    indexing_pipeline.connect("embedder", "writer")

    return indexing_pipeline


In [5]:
import wikipedia
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

some_bands = """The Beatles,The Cure""".split(",")

raw_docs = []

for title in some_bands:
    page = wikipedia.page(title=title, auto_suggest=False)
    doc = Document(content=page.content, meta={"title": page.title, "url": page.url})
    raw_docs.append(doc)

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
document_store_with_embedded_metadata = InMemoryDocumentStore(embedding_similarity_function="cosine")

indexing_pipeline = create_indexing_pipeline(document_store=document_store)
indexing_with_metadata_pipeline = create_indexing_pipeline(
    document_store=document_store_with_embedded_metadata, metadata_fields_to_embed=["title"]
)

indexing_pipeline.run({"cleaner": {"documents": raw_docs}})
indexing_with_metadata_pipeline.run({"cleaner": {"documents": raw_docs}})


/Users/ngohongthai/miniconda3/envs/nmt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 17/17 [00:12<00:00,  1.40it/s]


{'writer': {'documents_written': 538}}

### Comparing Retrieval With and Without Embedded Metadata

In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(model="thenlper/gte-large"))
retrieval_pipeline.add_component(
    "retriever", InMemoryEmbeddingRetriever(document_store=document_store, scale_score=False, top_k=3)
)
retrieval_pipeline.add_component(
    "retriever_with_embeddings",
    InMemoryEmbeddingRetriever(document_store=document_store_with_embedded_metadata, scale_score=False, top_k=3),
)

retrieval_pipeline.connect("text_embedder", "retriever")
retrieval_pipeline.connect("text_embedder", "retriever_with_embeddings")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - retriever_with_embeddings: InMemoryEmbeddingRetriever
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - text_embedder.embedding -> retriever_with_embeddings.query_embedding (List[float])

In [7]:
result = retrieval_pipeline.run({"text_embedder": {"text": "Have the Beatles ever been to Bangor?"}})

print("Retriever Results:\n")
for doc in result["retriever"]["documents"]:
    print(doc)

print("Retriever with Embeddings Results:\n")
for doc in result["retriever_with_embeddings"]["documents"]:
    print(doc)


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.10s/it]

Retriever Results:

Document(id=93be65bda2feadec9f9a36506fa2cc482036b4e330ad45e87cfa0115a14d1d68, content: ' The band flew to Florida, where they appeared on The Ed Sullivan Show a second time, again before 7...', meta: {'title': 'The Beatles', 'url': 'https://en.wikipedia.org/wiki/The_Beatles', 'source_id': '381a557f12a53098e0122374cc3e41bc0890b55b5954a84b2fdd1bf663853478', 'page_number': 1}, score: 0.8637015562978234)
Document(id=a6c1453a05fa0b322d266a172fac1d4402b88a48d93420cf1ddeb50f99b1a339, content: '
During the 1964 US tour, the group were confronted with racial segregation in the country at the ti...', meta: {'title': 'The Beatles', 'url': 'https://en.wikipedia.org/wiki/The_Beatles', 'source_id': '381a557f12a53098e0122374cc3e41bc0890b55b5954a84b2fdd1bf663853478', 'page_number': 1}, score: 0.8558952686555173)
Document(id=11ffff8f0c3ea8d0bc5c8f234cd20a23eb879e2950f5ac0947e6eb9ca250bb9d, content: 'The Beatles were an English rock band formed in Liverpool in 1960, comprising John L